In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tkinter import *
import datetime as dt
import numpy as np

province_name = input("Enter the name of your province: ")

# Options for the options menu
# Omitted "Repatriated" and "Not Reported" from the Options List
# Options menu simply here to test code
if province_name == "Alberta":
    OptionList = ["Calgary","Central","Edmonton",
                  "North","South",]

elif province_name == "British Columbia":
    province_name = "BC"
    OptionList = ["Fraser","Interior","Island",
                  "Northern","Vancouver Coastal",]

elif province_name == "Manitoba":
    OptionList = ["Interlake-Eastern","Northern",
                  "Prairie Mountain","Southern Health","Winnipeg",]

elif province_name == "New Brunswick":
    OptionList = ["Zone 1 (Moncton area)","Zone 2 (Saint John area)",
                  "Zone 3 (Fredericton area)","Zone 4 (Edmundston area)",
                  "Zone 5 (Campbellton area)","Zone 6 (Bathurst area)","Zone 7 (Miramichi area)",]

elif province_name == "Newfoundland and Labrador":
    province_name = "NL"
    OptionList = ["Central","Eastern","Labrador-Grenfell",
                  "Western",]

elif province_name == "Nova Scotia":
    OptionList = ["Zone 1 - Western","Zone 2 - Northern",
                  "Zone 3 - Eastern","Zone 4 - Central",]

elif province_name == "Nunavut":
    OptionList = ["Nunavut",]
    
elif province_name == "Northwest Territories":
    province_name = "NWT"
    OptionList = ["NWT",]

elif province_name == "Ontario":
    OptionList = ["Algoma","Brant","Chatham-Kent","Durham","Eastern","Grey Bruce","Haldimand-Norfolk", 
                  "Haliburton Kawartha Pineridge","Halton","Hamilton","Hastings Prince Edward",
                  "Huron Perth","Kingston Frontenac Lennox & Addington","Lambton",
                  "Leeds Grenville and Lanark","Middlesex-London","Niagara","North Bay Parry Sound",
                  "Northwestern","Ottawa","Peel","Peterborough","Porcupine",
                  "Renfrew","Simcoe Muskoka","Southwestern","Sudbury","Thunder Bay","Timiskaming",
                  "Toronto","Waterloo","Wellington Dufferin Guelph","Windsor-Essex","York",]

elif province_name == "Prince Edward Island":
    province_name = "PEI"
    OptionList = ["Prince Edward Island",]
    
elif province_name == "Quebec":
    OptionList = ["Abitibi-Témiscamingue","Bas-Saint-Laurent","Capitale-Nationale",
                  "Chaudière-Appalaches","Côte-Nord","Estrie","Gaspésie-Îles-de-la-Madeleine",
                  "Lanaudière","Laurentides","Laval","Mauricie","Montérégie",
                  "Montréal","Nord-du-Québec","Nunavik",
                  "Outaouais","Saguenay","Terres-Cries-de-la-Baie-James",]
    
elif province_name == "Saskatchewan":
    OptionList = ["Central","Far North","North",
                  "Regina","Saskatoon","South",]

elif province_name == "Yukon":
    OptionList = ["Yukon",]

def selected(event):
    MyLabel = Label(win, text="The selected region is: "+clicked.get()).pack()   
    
# Creates the window
win = Tk()

# Sets size of the window
win.geometry("400x400")

# Sets title of the window
win.title("COVID-19 Mortality")

clicked = StringVar()
clicked.set(OptionList[0])

# Creates the Option Menu
dropmenu = OptionMenu(win, clicked, *OptionList,command=selected)
dropmenu.pack(padx=10,pady=20)

win.mainloop()

print("The name of your region is: "+clicked.get())

weather_base_url = 'https://dd.weather.gc.ca/climate/observations/daily/csv/'
static_info = pd.read_csv(r'C:\Users\shafi\Downloads\COVID-19 CANADIAN DASHBOARD\hr_static_data.csv')

past = dt.datetime(2015, 1, 1)
start_date = past.strftime('%Y-%m-%d')

today = dt.datetime(2020, 12, 31)
end_date = today.strftime('%Y-%m-%d')

def climateid(province_name, region_name):
    static_info_province = static_info[static_info.province_name == province_name]
    return static_info_province.climate_id[static_info_province.health_region == region_name].item()

def provinceid(province_name, region_name):
    static_info_province = static_info[static_info.province_name == province_name]
    return static_info_province.prov_id[static_info_province.health_region == region_name].item()

def get_temp_dates(temp_files):
    dates = []
    for file in temp_files:
        weat_data =  pd.read_csv(file, encoding='Latin-1')
        weat_data['Date/Time'] = pd.to_datetime(weat_data['Date/Time'], errors='coerce')
        formatted_dates = weat_data['Date/Time'].dt.strftime('%m-%d')
        temp_dates = formatted_dates.values
        dates.extend(temp_dates)

    return dates

def get_temp_vals(temp_files):
    temps = []
    # temps = np.arange(10)
    for file in temp_files:
        weat_data =  pd.read_csv(file, encoding='Latin-1')
        temp_dates = weat_data['Mean Temp (°C)'].values
        temps.extend(temp_dates)
    return temps

def get_temp_files(province_name, region_name, start_date, end_date):
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
    num_months = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1
    temp_files = []

    prov_id = provinceid(province_name, region_name)
    climate_id = climateid(province_name, region_name)

    for year in range(start_date.year, end_date.year+1):
        year = str(year)
        for month in range(1,13):
            if month in range(1,10):
                month = '0'+str(month)
            else:
                month = str(month)

            gloabl_date = str(year) + "-" + month 
            target_url = weather_base_url + prov_id + '/climate_daily_' + prov_id + '_' + climate_id + '_' + gloabl_date + '_P1D.csv'
            temp_files.append(target_url)

    return temp_files

temp_files = get_temp_files(province_name, clicked.get(), start_date, end_date)

data = {'Date':  get_temp_dates(temp_files),
        'Mean Temperature': get_temp_vals(temp_files)}

df_weat = pd.DataFrame(data, columns = ['Date','Mean Temperature'])

# Arbitrary date for avg_temp_data function
# The date itself doesn't matter since it's only used to loop
# through all the days of a year (including a leap year)
begin_year = dt.date(2015, 12, 31)
end_year = dt.date(2016, 12, 31)

def avg_temp_data(begin_year, end_year):
    
    one_day = dt.timedelta(days=1)

    next_day = begin_year
    for day in range(0, 366):  # Includes leap year
        if next_day > end_year:
            break
        
        # Adds a day to the current date
        next_day += one_day
        date_range = next_day.strftime('%m-%d')
        df_weat_date = df_weat.groupby('Date')['Mean Temperature'].mean()
    return df_weat_date

print(avg_temp_data(begin_year, end_year))

Enter the name of your province: Ontario
The name of your region is: Toronto
Date
01-01   -2.866667
01-02   -1.650000
01-03   -0.466667
01-04   -3.500000
01-05   -6.260000
           ...   
12-27   -0.183333
12-28   -0.950000
12-29   -1.266667
12-30   -0.800000
12-31   -2.200000
Name: Mean Temperature, Length: 366, dtype: float64
